In [ ]:
# 0) При реализации алгоритма разрешается использовать только библиотеки из requierments.txt

В него входит:
1. jupyter - библиотека для показа ноутбуков
2. numpy - библиотека для вычислений
3. matplotlib - библиотека для визуализации

## Установка 

1. Устанавливаем python3 и virtualenv
2. создаем окружение virtualenv --no-site-packages  lin_prog
3. активируем окружение source activate lin_prog
4. устанавливаем зависимости pip install -r requirements.txt
5. запускаем jupyter и начинаем работать jupyter notebook



# ==========================================================
# Задача на МНК (0.4 балла)

In [3]:
from math import sin
import numpy as np

"""Пусть физический закон описывается зависимостью 
некоторого измеряемого значения y(x, a) 
от времени и координаты x при параметрах a:"""
def y(t,a): 
    return a[2]*sin(t)+a[1]*t +a[0]
    
"""
Дан набор координат t размера m, значения распределены равномерно). Пусть m = 200.
"""
m=200
t=[i*10.0/m for i in range(m)]


"""Для каждого момента времени t сгенерируйте соответствующее 
значение y(t,a) при некоторых параметрах  a_0, a_1, a_2. Для примера: """
a=[10,100,1000]

def get_y (a, σ):
    """Результаты измерений отличаются от истинных значений в силу действия случайной аддитивной помехи 
    (случайность подчиняется нормальному закону распределения N(0, σ))"""
    y_real=np.array([y(i,a) for i in t])
    y_corr=y_real+np.random.normal(0,σ,m)
    return y_real, y_corr

#todo -выбрать параметр
σ=0.5

#генерация значений. изначальные и с помехами
y_real, y_corr= get_y(a,σ)


def get_params (y_corr, t, method=0):
    """
    По сгенерированному набору точек y_corr дайте оценку параметрам a
    закона с учетом знания общей формулы тремя различными способами:
    •	method=0 -> сумма квадратов невязок будет минимальна.
    •	method=1 -> сумма абсолютных значений невязок будет минимальна.
    •	method=2 -> максимальное абсолютное значение невязки будет минимально.
    
    #todo - написать ф-ю
    """
    A = np.vstack([x, np.ones(len(x))]).T
    return np.linalg.lstsq()
    
    
    return [0,0,0]

## Задание 1 (0.2 балла)
1.	Постройте в одной координатной плоскости графики у(t, a) и  оценочные значения y(t,a*) для всех 3 методов

2.	Вычислите как отличается каждый из оценочных параметров от своего истинного значения. Как меняется это отличие при изменении σ?
3.	Скорректируйте y_corr[0]  и y_corr[-1] пусть одно из них будет на 50 больше, а другое на 50 меньше. Постройте новые оценочные значения параметров  и соответствующие графики. Какая из оценок получилась более устойчивой к выбросам?

## Задание 2 (0.2 балла)
Возьмем случайную матрицу A  200x80 и случайный вектор b из распределения N(0,1). 

1. Решите переопределенную систему тремя способами, минимизируя l1, l2 и linf нормы вектора b – Ax.
2. Постройте распределение ошибок для каждого решения.
3. Какими свойствами обладают распределения?

In [4]:
sample = np.random.normal(size=(200, 80))


# ============================================================
# Задача на Симплекс метод

## 1) На вход Вашему функцию должны приходить:

0. число переменных = n
1. матрица A (n x m) (tsv, вещественные числа)
2. вектор  b  ограничений типа неравнство
4. вектор c функции полезности для задачи max cx 
5. алгоритм выбора входящей переменной (правило Бленда, Лексикографический метод)
6. (не обязательный параметр) стартовую базисную точку

## 2) На выход программа должна выдавать:

### Обязательная часть (0.3 баллов):
0. Ответ и оптимальную точку при положительных компонентах вектора b
1. Количество итераций потребовавшихся для решения задачи
2. при n=2 выдавать процесс решения (draw=True)
3. Напишите программу которая будет отвечать на вопрос оптимально ли приведенное решение, например


### Дополнительная часть  (0.8 балл):
0. Максимально использовать матричные вычисления (0.2 балла)
1. Работать в случае отрицательных чисел в векторе b (0.2 балла)




In [129]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

#пример из листочка 1
A=np.array([[1,2],[2,0.5], [3, 14]])
b=np.array([5,8,4])
c=np.array([5,1])
print(A,A.shape,b,c)

A=np.array([[1,-1],[2,-1], [0, 1]])
b=np.array([1,3,5])
c=np.array([4,3])

A=np.array([[2,3,1],[4,1,2],[3,4,2]])
b=np.array([5, 11, 8])
c=np.array([5, 4, 3])

[[  1.    2. ]
 [  2.    0.5]
 [  3.   14. ]] (3, 2) [5 8 4] [5 1]


In [130]:
import numpy as np
import logging as log
import sys
log.basicConfig(level=log.INFO, format='>%(message)s', stream=sys.stdout)

def solve_lin_prog (A, b, c, method='blend', start_point=None, draw=False):
    """
    Здесь должно быть ваше решение. У всех действий должны быть комментарии. 
    Код должен быть читабельным, хорошо использовать дополнительные функции если это необходимо
    
    A, b, c - матрица, b - вектор ограничений типа <=, с - функция полезности, задача максимизации
    method - 'blend', 'lexical'
    start_point - точка
    draw -  true/false рисовать ли ответ, только для 2 переменных
    
    Вывод - вектор на котором достигается максимум, максимальное значение, число итераций
    """
    
    # Если есть отрицательные значения в b, запускаем инициализацию
    if np.count_nonzero(b < 0) > 0:
        pass
    
    # Инициализируем матрицы и прочее
    N = A
    B = np.identity(n=N.shape[0])
    A = np.hstack((B, N))
    print('A initial', A)
    n = N.shape[0]
    m=N.shape[1]
    print('n=%s, m=%s'%(n, m))
    
    c_n = c.reshape((-1, 1))
    c_b = np.zeros(n).reshape((-1, 1))
    c = np.vstack((c_b, c_n))
    print('c initial', c, c_b, c_n, sep='\n')
    
    b = b.reshape((-1, 1))
    
    x_b = b
    x_n = np.zeros_like(c_n)
    x = np.vstack((x_b, x_n))
    print('x initial',x, x_b, x_n, sep='\n')
    
    B_set = set(range(0, B.shape[0]))
    print(B_set)
    
    
    def check_if_optimal(c):
        return np.max(c) <= 0
    
    def choose_entering(c):
        #feasible_set = np.argwhere(c > 0)
        return np.argmax(c)
    
    def e_j(n, j):
        ret = np.zeros(n)
        ret[j] = 1
        return ret.reshape((-1, 1))
    
    def choose_leaving(delta_x_b):
        pass
    
    
    iterations_count = 1
    # Главный цикл!
    while not np.count_nonzero(b < 0) > 0:
        print('        ITERATION #%s'% iterations_count)
        iterations_count += 1
        j = choose_entering(c)-n
        print('j=', j)


        delta_x_b = np.linalg.inv(B)@N@e_j(m, j)
        print('delta_x_b', delta_x_b)
        ratios = delta_x_b/b#x[:m+1]
        #print(ratios)
        i = np.argmax(ratios)
        print('i=',i)
        t = (1/ratios[i])[0]
        print('t=',t)

        delta_c_n = (np.linalg.inv(B)@N).T@e_j(n, i)
        print('delta_c_n', delta_c_n)
        s = (c[j+n]/delta_c_n[j])[0]
        print('s=', s)

        x[j] = t
        x[:n] = x[:n] - t*delta_x_b
        print('x',x)

        c[i+n] = s
        c[n:] -= s*delta_c_n
        print('c',c)

        B_set.remove(i)
        B_set.add(j+n)
        print(B_set)

        print('Пересчёт')
        # Пересчитаем сразу всё для следующей итерации
        basis_ind = sorted(list(B_set))
        nonbasis_ind = sorted(list(set(range(0, m+n))-B_set))
        B_ = A.T[basis_ind].T
        N_ = A.T[nonbasis_ind].T
        A_ = np.hstack((B, N))
        print(A_, B_, N_, sep='\n')
        print('try', np.linalg.inv(B)@B_, sep='\n')
        #n = N.shape[0]
        #m = N.shape[1]
        #print('n=%s, m=%s'%(n, m))

        c_n = c.T[0, nonbasis_ind].reshape(-1, 1)
        c_b = c.T[0, basis_ind].reshape(-1, 1)
        c = np.vstack((c_b, c_n))
        print('c', c, c_b, c_n, sep='\n')

        x_b = x.T[0, basis_ind].reshape(-1, 1)
        x_n = x.T[0, nonbasis_ind].reshape(-1, 1)
        x = np.vstack((x_b, x_n))
        print('x', x, x_b, x_n, sep='\n')
    
    
    
    
    #x=np.array([4,0])
    #result=20
    #num_iter=1
    
    #тут рисуем анимацию
    if draw:
        fig, ax= plt.subplots(num_iter+1)
        fig.set_figheight(5*(num_iter+1))
        fig.set_figwidth(5)
        xs=[[0,0],[4,0]]

        for i,a in enumerate(ax):
            a.plot([0,5],[5,0],  color='b')
            a.plot([4,0],[0,16], color='b')
            a.plot([0,0],[0,5], color='b')
            a.plot([0,4],[0,0], color='b')
            a.axis([-1, 17, -1, 17])
            a.set_xlabel('X1')
            a.set_ylabel('X2')
            a.set_title('Iteration %d x=(%.2f, %.2f)' % (i+1,xs[i][0],xs[i][1]))

        ax[0].scatter([0,4,3.66],[5,0,1.33], color='black')
        ax[0].scatter([0],[0], color='red')

        ax[1].scatter([0,0,3.66],[0,5,1.33], color='black')
        ax[1].scatter([4],[0], color='red')


        plt.tight_layout()
        plt.show()
    
    #return np.array([4,0]), 20, 1
    return x, 0, 0


def is_optimal (A,b,c, x):
    """
    Здесь должна быть реализована проверка оптимальности точки. 
    Алгоритм должен работать для фиксированных n,m за константное время
    """
    return (x==np.array([4,0])).all()

In [150]:


def solve_lin_prog (A, b, c, method='blend', start_point=None, draw=False):
    """
    Здесь должно быть ваше решение. У всех действий должны быть комментарии. 
    Код должен быть читабельным, хорошо использовать дополнительные функции если это необходимо
    
    A, b, c - матрица, b - вектор ограничений типа <=, с - функция полезности, задача максимизации
    method - 'blend', 'lexical'
    start_point - точка
    draw -  true/false рисовать ли ответ, только для 2 переменных
    
    Вывод - вектор на котором достигается максимум, максимальное значение, число итераций
    """
    
    # Если есть отрицательные значения в b, запускаем инициализацию
    if np.count_nonzero(b < 0) > 0:
        pass
    
    # Инициализируем матрицы и прочее
    
    b = b.reshape((-1, 1))
    
    top_line = np.hstack((0, np.zeros(A.shape[1]), 0))
    mid = np.hstack((b, A, b))
    bottom_line = np.hstack((0, -c, 0))
    
    ST = np.vstack([top_line, mid, bottom_line])
    
    n = ST.shape[0]
    m = ST.shape[1]
    print('n=%s, m=%s'%(n, m))
    
    basis = list(range(0, n-2))
    nonbasis = list(range(n-2, (n-2)+(m-2)))
    print('basis', basis, 'nonbasis', nonbasis)

    iteration_number = 0
    
    while np.min(ST[n-1]) < 0:
        iteration_number += 1
        print('          ITERATION #%s'%iteration_number)
        print(ST)
        
        #Разрешающий столбец
        print(ST[-1])
        j = np.argmin(ST[-1][1:-1])+1
        print('j=', j)

        ratios = ST[:, -1]/ST[:, j]
        print(ratios)
        i = np.argmin(ratios[1:-1])+1#[:-1]
        print('i=', i)

        #Строим новую таблицу
        basis[i-1], nonbasis[j-1] = nonbasis[j-1], basis[i-1]
        print('basis', basis, 'nonbasis', nonbasis)
        ST[i][j] = 1/ST[i][j]
        print(ST)
        ST[i][np.arange(len(ST[i])) != j] /= ST[i][j]
        print(ST)
        ST[:, j][np.arange(len(ST[:,j])) != i] /= -ST[i][j]
        print(ST)

        for i_ in range(1, n-1):
            for j_ in range(1, m-1):
                print(i_, j_)
                if i_ != i and j_ != j:
                    ST[i_][j_] -= (ST[i][j]*ST[i_][j_]-ST[i_][j]*ST[i][j_])/ST[i][j]

        print(ST)
    
    print('FINISHED in %d iterations'%iteration_number)
    
    x = np.zeros(n)
    for i in basis:
        if 0<i<len(c):
            x[i] = ST[i+1][0]
    for i in nonbasis:
        if 0<i<len(c):
            x[i] = ST[0][i+1]
    
    #return np.array([4,0]), 20, 1
    return x, 0, 0


In [151]:
solve_lin_prog(A, b, c)

n=5, m=5
basis [0, 1, 2] nonbasis [3, 4, 5]
          ITERATION #1
[[  0.   0.   0.   0.   0.]
 [  5.   2.   3.   1.   5.]
 [ 11.   4.   1.   2.  11.]
 [  8.   3.   4.   2.   8.]
 [  0.  -5.  -4.  -3.   0.]]
[ 0. -5. -4. -3.  0.]
j= 1
[        nan  2.5         2.75        2.66666667 -0.        ]
i= 1
basis [3, 1, 2] nonbasis [0, 4, 5]
[[  0.    0.    0.    0.    0. ]
 [  5.    0.5   3.    1.    5. ]
 [ 11.    4.    1.    2.   11. ]
 [  8.    3.    4.    2.    8. ]
 [  0.   -5.   -4.   -3.    0. ]]
[[  0.    0.    0.    0.    0. ]
 [ 10.    0.5   6.    2.   10. ]
 [ 11.    4.    1.    2.   11. ]
 [  8.    3.    4.    2.    8. ]
 [  0.   -5.   -4.   -3.    0. ]]
[[  0.   -0.    0.    0.    0. ]
 [ 10.    0.5   6.    2.   10. ]
 [ 11.   -8.    1.    2.   11. ]
 [  8.   -6.    4.    2.    8. ]
 [  0.   10.   -4.   -3.    0. ]]
1 1
1 2
1 3
2 1
2 2
2 3
3 1
3 2
3 3
[[  0.   -0.    0.    0.    0. ]
 [ 10.    0.5   6.    2.   10. ]
 [ 11.   -8.  -96.  -32.   11. ]
 [  8.   -6.  -72.  -24.    8.

(array([  0.00000000e+00,  -0.00000000e+00,  -1.95748513e+44,
          0.00000000e+00,   0.00000000e+00]), 0, 0)

In [141]:
x, best, n_iter = solve_lin_prog(A,b,c, draw=True)

n=5, m=5
basis [0, 1, 2] nonbasis [3, 4, 5]
          ITERATION #1
[[  0.   0.   0.   0.   0.]
 [  5.   2.   3.   1.   5.]
 [ 11.   4.   1.   2.  11.]
 [  8.   3.   4.   2.   8.]
 [  0.  -5.  -4.  -3.   0.]]
[ 0. -5. -4. -3.  0.]
j= 1
[        nan  2.5         2.75        2.66666667 -0.        ]
i= 1
basis [3, 1, 2] nonbasis [0, 4, 5]
[[  0.    0.    0.    0.    0. ]
 [  5.    0.5   3.    1.    5. ]
 [ 11.    4.    1.    2.   11. ]
 [  8.    3.    4.    2.    8. ]
 [  0.   -5.   -4.   -3.    0. ]]
[[  0.    0.    0.    0.    0. ]
 [ 10.    0.5   6.    2.   10. ]
 [ 11.    4.    1.    2.   11. ]
 [  8.    3.    4.    2.    8. ]
 [  0.   -5.   -4.   -3.    0. ]]
[[  0.   -0.    0.    0.    0. ]
 [ 10.    0.5   6.    2.   10. ]
 [ 11.   -8.    1.    2.   11. ]
 [  8.   -6.    4.    2.    8. ]
 [  0.   10.   -4.   -3.    0. ]]
1 1
1 2
1 3
2 1
2 2
2 3
3 1
3 2
3 3
[[  0.   -0.    0.    0.    0. ]
 [ 10.    0.5   6.    2.   10. ]
 [ 11.   -8.  -96.  -32.   11. ]
 [  8.   -6.  -72.  -24.    8.

In [7]:
print (u'Точка: ', x)
print (u'Ответ: ', best)
print (u'Число итераций: ', n_iter)

Точка:  [4 0]
Ответ:  20
Число итераций:  1


In [8]:
is_optimal(A,b,c,x)

True

## Бонус +1 Балл
Напишите программу которая для обоих методов из задачи 5 будет использовать 2^n-1 итераций (бонус за каждый метод) и напишите обоснование (итого 0.5 балла за каждый метод)